In [1]:
import pandas as pd

In [2]:
matches = pd.read_csv('../data/train.csv')

In [3]:
matches.head(4)

,match_id,color,rank,map_code,duration,car_name,possession_time,time_in_side,shots,shots_against,...,percent_defensive_half,percent_offensive_half,percent_behind_ball,percent_infront_ball,percent_most_back,percent_most_forward,percent_closest_to_ball,percent_farthest_from_ball,demos_inflicted,demos_taken
0,0,blue,silver,neotokyo_standard_p,163,Breakout,54.95,58.96,2,4,...,62.736664,37.263340,66.84289,33.157120,95.53408,95.53408,95.53408,95.53408,1,0
1,0,orange,silver,neotokyo_standard_p,163,Octane,27.51,80.68,4,2,...,63.576576,36.423428,77.84667,22.153326,98.30417,98.30417,98.30417,98.30417,0,1
2,1,blue,gold,stadium_foggy_p,460,Octane,132.98,244.72,7,10,...,75.199120,24.800879,73.99232,26.007679,96.94244,96.94244,96.94244,96.94244,0,0
3,1,orange,gold,stadium_foggy_p,460,Octane,102.55,148.84,10,7,...,55.832005,44.167990,77.79280,22.207201,96.91851,96.91851,96.91851,96.91851,0,0


**Goal:** Predict the rank based on the total shots.

In [4]:
matches_prepped = matches.groupby(['match_id', 'rank'])['shots'].sum().reset_index()
matches_prepped.head()

,match_id,rank,shots
0,0,silver,6
1,1,gold,17
2,2,silver,3
3,3,platinum,12
4,4,platinum,16


In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [19]:
variables = ['shots']

X = matches_prepped[variables]
y = matches_prepped['rank']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 321)

logreg = LogisticRegression().fit(X_train, y_train)

y_pred = logreg.predict(X_test)

C:\Users\jrior\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [7]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [8]:
accuracy_score(y_test, y_pred)

0.29106360377107954

In [9]:
confusion_matrix(y_test, y_pred)

array([[   0,    6,   10,   85,   91,    0],
       [   0,  586,  265,  110,  484,    0],
       [   0,  439,  319,  186,  878,    0],
       [   0,  138,  168,  282,  939,    0],
       [   0,  309,  280,  271, 1005,    0],
       [   0,   42,   60,  165,  413,    0]], dtype=int64)

In [10]:
print(classification_report(y_test, y_pred, zero_division = 0))

              precision    recall  f1-score   support

      bronze       0.00      0.00      0.00       192
    champion       0.39      0.41      0.40      1445
     diamond       0.29      0.18      0.22      1822
        gold       0.26      0.18      0.21      1527
    platinum       0.26      0.54      0.35      1865
      silver       0.00      0.00      0.00       680

    accuracy                           0.29      7531
   macro avg       0.20      0.22      0.20      7531
weighted avg       0.26      0.29      0.26      7531



Now that I'm happy with the results, I want to refit the model on all of the available training data.

In [11]:
logreg = LogisticRegression().fit(X, y)

In [12]:
test = pd.read_csv('../data/test.csv')

In [13]:
test = test.groupby('match_id')['shots'].sum().reset_index()

In [14]:
y_pred = logreg.predict(test[variables])

In [15]:
converter = { 'bronze': 1, 'silver': 2, 'gold': 3, 'platinum': 4, 'diamond': 5, 'champion': 6 }

y_pred = pd.Series(y_pred).map(converter)

In [16]:
submission = pd.concat([test['match_id'], y_pred], axis = 1).rename(columns = {0: 'rank'})

In [17]:
submission.to_csv('../submissions/submission_2022-03-22_v1.csv', index = False)

In [20]:
submission

,match_id,rank
0,30121,5
1,30122,3
2,30123,4
3,30124,3
4,30125,4
...,...,...
2495,32616,4
2496,32617,3
2497,32618,4
2498,32619,4
